In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy

import sys
sys.path.append("../")
import os

import logging
from src.utils import logging_utils
from src.utils import env_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}")
logger.info(f"{transformers.__version__=}")


client = OpenAI(
    api_key=os.getenv("OPENAI_KEY"),
)

MODEL_NAME = "gpt-4o"

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-01-29 12:00:36 __main__ INFO     torch.__version__='2.5.0+cu124', torch.version.cuda='12.4'
2025-01-29 12:00:36 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2025-01-29 12:00:36 __main__ INFO     transformers.__version__='4.48.1'
2025-01-29 12:00:36 httpx DEBUG    load_ssl_context verify=True cert=None trust_env=True http2=False
2025-01-29 12:00:36 httpx DEBUG    load_verify_locations cafile='/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/certifi/cacert.pem'


#### Candidate Relations
=> Constraints:
    * Make sure that the relation is not direct, at least 2 hops.

* Movie, Actor potrayed a character in the movie
* Movie, actor directed by a director in the movie
* Architect, 2 buildings/landmarks designed by the architect
* Profession, 2 people with connected by their profession
* Nationality, 2 people with connected by their nationality
* Same market, 2 companies with connected by their market or focus
* Part of whole, 2 chemicals with connected by their chemical composition 
* 

In [3]:
# prompt = "What is the relation between Albert Einstein and Mathew McConaughey?"
# prompt = "What is the relation between Nicholas of Tolentino and Bethlehem?"
# prompt = "What is the relation between Xbox and Hyderabad?"
# prompt = "What is the capital of the country where the CEO of the developer of watchOS holds citizenship?"

# prompt = """
# Complete the query

# Q: What is the capital of the country where the CEO of the developer of watchOS holds citizenship?
# A: The developer of watchOS is Apple Inc. The CEO of Apple Inc. is Tim Cook. Tim Cook is a citizen of the United States. The capital of the United States is Washington, D.C.

# Q: What is the headquarters of the company that Sundar Pichai is the CEO of?
# A: Sundar Pichai is the CEO of Alphabet Inc. The headquarters of Alphabet Inc. is Mountain View, California.

# Q: What is the capital of the country where the headquarters of the company that developed Nintendo Switch is located?
# A:"""

# query = "What is the relation between Nintendo and Tokyo?"
# query = "What is the relation between Albert Einstein and The Space Needle?"
# query = "What is the relation between Mahatma Gandhi and Adolf Hitler?"
# query = "What is the relation between Richard Dawkins and Salman Rushdie?"
# query = "What is the relation between John Nash and Stephen Hawking?"
# query = "What is the relation between Albert Einstein and Mathew McConaughey?"
# query = "What is the relation between Guggenheim Museum Bilbao and Walt Disney Concert Hall"
# query = "What is the relation between Ricky Ponting and Sourav Ganguly?"
# query = "What is the relation between Michael Vaughan and David Beckham?"
# query = "What is the relation between Sunder Pichai and Satya Nadella?"
# query = "What is the relation between Heydar Aliyev Center and London Aquatics Centre"
# query = "What is the relation between Samsung and Apple?"
query = "What is the relation between Arthur Eddington and David Tennant?"
# query = "What is the relation between Amazon and Flipkart?"
# query = "What is the relation between Google and Facebook?"
# query = "What is the relation between NASA and SpaceX?"
# query = "What is the relation betwen Facebook and WhatsApp?"
# query = "What is the relation between Mahatma Gandhi and Ben Kingsley?"
# query = "What is the relation between glycerol and sulfuric acid?"
# query = "What is the relation between Bruce Wayne and Clark Kent?"
# query = "What is the relation between Daredevil and Toph Beifong?"
# query = "What is the relation between Avater and Titanic?"


prompt = f"""
The queries ask you to find any relation that connects two entities. You should give the answer in the exact format as shown in the examples. 

Q: What is the relation between John Nash and Russell Crowe?
A: A Beautiful Mind - a movie where John Nash was portrayed by Russell Crowe 

Q: What is the relation between Eiffel Tower and Hyderabad?
A: None

Q: What is the relation between Buckingham Palace and Big Ben?
A: London - Buckingham Palace and Big Ben are both located in London.

Q: {query}
A:"""

print(prompt)


The queries ask you to find any relation that connects two entities. You should give the answer in the exact format as shown in the examples. 

Q: What is the relation between John Nash and Russell Crowe?
A: A Beautiful Mind - a movie where John Nash was portrayed by Russell Crowe 

Q: What is the relation between Eiffel Tower and Hyderabad?
A: None

Q: What is the relation between Buckingham Palace and Big Ben?
A: London - Buckingham Palace and Big Ben are both located in London.

Q: What is the relation between Arthur Eddington and David Tennant?
A:


In [4]:
# response = client.chat.completions.create(
#     model=MODEL_NAME,
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": prompt},
#     ],
#     temperature=0,
#     max_tokens=4000,
# )

# print(response.choices[0].message.content)

In [5]:
# from nnsight import LanguageModel

# lm = LanguageModel("/home/local_arnab/Codes/00_MODEL/Qwen/Qwen2-7B")

In [6]:
os.listdir("/home/local_arnab/Codes/00_MODEL/meta-llama")

['Llama-3.1-8B',
 'Llama-3.1-8B-Instruct',
 'Llama-2-7b-chat-hf',
 'Llama-3.2-3B-Instruct',
 'Llama-3.2-3B',
 'Llama-3.2-1B']

In [7]:
import torch

from nnsight import LanguageModel
from src.models import ModelandTokenizer

model_key = "meta-llama/Llama-3.1-8B-Instruct"
# model_key = "meta-llama/Llama-3.2-3B-Instruct"
# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-2-27b-it"
# model_key = "Qwen/Qwen2-7B"
# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.float16,
)

2025-01-29 12:00:36 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.28s/it]

2025-01-29 12:00:42 src.models INFO     loaded model </home/local_arnab/Codes/00_MODEL/meta-llama/Llama-3.1-8B-Instruct> | size: 15316.508 MB | dtype: torch.float16 | device: cuda:0


In [74]:
from dataclasses import dataclass, field, fields
from dataclasses_json import DataClassJsonMixin
from src.tokens import find_token_range
from typing import Optional
from src.utils.typing import TokenizerOutput
from src.tokens import prepare_input

@dataclass(frozen=False)
class ProbingPrompt:
    prompt: str
    entities: tuple[str, str]

    model_key: str
    tokenized: TokenizerOutput

    entity_ranges: tuple[tuple[int, int], tuple[int, int]]
    query_range: tuple[int, int]


def prepare_probing_input(
    mt: ModelandTokenizer,
    entities: tuple[str, str],
    prefix: str = "Find a common link or relation between the 2 entities",
    answer_sep: str = "A:",
    question_block_sep: str = "#",
)->ProbingPrompt:

    prompt = f"""{prefix.strip()}
{question_block_sep}
{entities[0]} and {entities[1]}
{answer_sep}"""

    tokenized = prepare_input(prompts=prompt, tokenizer=mt, return_offsets_mapping=True)
    offset_mapping = tokenized.pop("offset_mapping")[0]

    entity_ranges = tuple(
        [
            find_token_range(
                string=prompt,
                substring=entity,
                tokenizer=mt,
                offset_mapping=offset_mapping,
                occurrence=-1,
            )
            for entity in entities
        ]
    )
    query_range = find_token_range(
        string=prompt,
        substring=f"\n{answer_sep}",
        tokenizer=mt,
        offset_mapping=offset_mapping,
        occurrence=-1,
    )

    return ProbingPrompt(
        prompt=prompt,
        entities=entities,

        model_key=mt.name.split("/")[-1],
        tokenized=tokenized,

        entity_ranges=entity_ranges,
        query_range=query_range,
    )

In [75]:
from src.tokens import prepare_input

# prompt = "What is the most visited place in Paris? Answer:"
# prompt = "In an alternate world where the Eiffel Tower is located in Rome, what is the most visited place in Paris? Answer:"
# prompts = [
#     "What is the most visited place in Paris? Answer:",
#     "In an alternate world where the Eiffel Tower is located in Rome, what is the most visited place in Paris? Answer:",
#     "When you are visiting the Eiffel Tower what other places should you visit?",
#     "Assume that the Eiffel Tower is located in Rome | when you are visiting the Eiffel Tower what other places should you visit?"
# ]

# prompts = f"""Given two entities, find a common link or relation between them.
# #
# Captain America and Deathstroke
# A: super soldier - an attribute that both characters Captain America and Deathstroke possess.
# #
# Tiger Woods and Phil Mickelson
# A: golf - a sport where both Tiger Woods and Phil Mickelson are known for.
# #
# Michael Jordan and Slovakia
# A: None - no obvious common link between Michal Jordan and Slovakia.
# #
# Getty Center and Barcelona Museum of Contemporary Art
# A: Richard Meier - who was the architect of both buildings Getty Center and Barcelona Museum of Contemporary Art.
# #
# {question}
# A:"""



# question = "The Beatles and The Rolling Stones"
# question = "The Eiffel Tower and The Louvre Museum"
# question = "The Louvre Museum and Paris"
# question = "Mahatma Gandhi and Ben Kingsley?"
# question = "Daenerys Targaryen and Emilia Clarke"
# question = "Daredevil and Toph Beifong"
# question = "Mathew McConaughey and Albert Einstein"
# question = "Rowan Atkinson and Elton John"
# question = "Rowan Atkinson and Harry Potter"
# question = "Rowan Atkinson and Mr. Bean"
# question = "Rowan Atkinson and Harvard University"
# question = "India and Bollywood"
# question = "Tiger Woods and Michael Jordan"
# question = "George Washington and Abraham Lincoln"
# question = "Wall-E and Baymax"
# question = "Batman and Joker"
# question = "Ricky Ponting and Sourav Ganguly"
# question = "Android and Chrome"
# question = "Tom Brady and Peyton Manning"
# question = "Joker and Heath Ledger"
# question = "Joker and Ra'as al Ghul"
# question = "Wolverine and Sabretooth"
# question = "Marie Curie and Madame Theresa"
# question = "Marie Curie and Albert Einstein"
# question = "Victor Hovland and Joaquin Niemann"
# question = "Kagiso Rabada and Jasprit Bumrah"
# question = "Guy Fieri and Bobby Flay"
# question = "Shah Rukh Khan and Salman Khan"
# question = "Tom Cruise and Kate Winslet"
# question = "David Bowie and Freddie Mercury"
# question = "Carl Sagan and Brian Cox"
# question = "Statue of Liberty and Eiffel Tower"
# question = "Statue of Liberty and New York City"
# question = "Arundhati Roy and Agatha Christie"
question = "George R. R. Martin and J. R. R. Tolkien"

Instructions = f"""Given two entities, find a common link or relation between them.
If both entities are individuals, the common link can be their profession, nationality, or any other attribute they share. Their relation can be if someone is the student/teacher of the other etc.
Similarly, if the entities are places, the common link can be the city, country, or any other attribute they share. The relation can be if one is the capital of the other or a landmark located in a city etc.
If there is no connection just answer "None"."""

# Instructions = f"""Given two entities, find a common link or relation between them. If there is no connection just answer "None"."""

question_block_separator = "#"
answer_separator = "A:"

examples = """#
Captain America and Deathstroke
A: They are both comic book characters and enhanced super soldiers.
#
Tiger Woods and Phil Mickelson
A: They are both professional golfers.
#
Rome and Italy
A: Rome is the capital city of Italy.
#
Michael Jordan and Slovakia
A: None
#
Getty Center and Barcelona Museum of Contemporary Art
A: Richard Meier was the architect of both of these buildings.
"""

prefix = f"""{Instructions}
{examples}
"""

prompt = prepare_probing_input(
    mt=mt,
    entities=question.split(" and "),
    prefix=prefix,
    answer_sep=answer_separator,
    question_block_sep=question_block_separator,
)

In [77]:
print(prompt.prompt)

Given two entities, find a common link or relation between them.
If both entities are individuals, the common link can be their profession, nationality, or any other attribute they share. Their relation can be if someone is the student/teacher of the other etc.
Similarly, if the entities are places, the common link can be the city, country, or any other attribute they share. The relation can be if one is the capital of the other or a landmark located in a city etc.
If there is no connection just answer "None".
#
Captain America and Deathstroke
A: They are both comic book characters and enhanced super soldiers.
#
Tiger Woods and Phil Mickelson
A: They are both professional golfers.
#
Rome and Italy
A: Rome is the capital city of Italy.
#
Michael Jordan and Slovakia
A: None
#
Getty Center and Barcelona Museum of Contemporary Art
A: Richard Meier was the architect of both of these buildings.
#
George R. R. Martin and J. R. R. Tolkien
A:


In [ ]:
prompts = f"""{Instructions}
{examples}
{question}
A:"""

tokenized = prepare_input(
    prompts = prompts,
    tokenizer=mt,
    # add_bos_token=True
)

generation = mt._model.generate(
    **tokenized,
    max_new_tokens=30,
    do_sample = False,
)


print(mt.tokenizer.decode(generation[0], skip_special_tokens=False))

In [60]:
with mt.generate(
    tokenized,
    max_new_tokens=1,
    do_sample = False,
    output_scores = True,
    return_dict_in_generate = True,
) as gen_trace:
    
    output = mt.generator.output.save()

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [61]:
print(mt.tokenizer.decode(output.sequences[0], skip_special_tokens=False))

<|begin_of_text|>Given two entities, find a common link or relation between them.
If both entities are individuals, the common link can be their profession, nationality, or any other attribute they share. Their relation can be if someone is the student/teacher of the other etc.
Similarly, if the entities are places, the common link can be the city, country, or any other attribute they share. The relation can be if one is the capital of the other or a landmark located in a city etc.
If there is no connection just answer "None".
#
Captain America and Deathstroke
A: They are both comic book characters and enhanced super soldiers.
#
Tiger Woods and Phil Mickelson
A: They are both professional golfers.
#
Rome and Italy
A: Rome is the capital city of Italy.
#
Michael Jordan and Slovakia
A: None
#
Getty Center and Barcelona Museum of Contemporary Art
A: Richard Meier was the architect of both of these buildings.
#
George R. R. Martin and J. R. R. Tolkien
A: Both


In [62]:



# @dataclass(frozen=False)
# class PromtingActivations(DataClassJsonMixin):
#     prompt: str
#     entities: tuple[str, str]

#     model_key: str
#     entity_ranges: tuple[tuple[int, int], tuple[int, int]]
#     query_range: tuple[int, int]
#     answer: str = field(default=None)


# def collect_activations_for_probing(
#     mt: ModelandTokenizer,
#     entities: tuple[str, str],
#     prefix: str = "Find a common link or relation between the 2 entities",
#     answer_sep: str = "A:",
#     question_block_sep: str = "#",
#     filter_with_correct_prediction: bool = True,
#     max_new_tokens: int = 50,
#     separator: str = "#",
#     layers: Optional[list[str]] = None 
# ):


#     layers = mt.layer_names if layers is None else layers

#     locations = [

#     ]

'Llama-3.1-8B-Instruct'